In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

In [2]:
import analyze_utils

# https://wandb.ai/jack-morris/emb-inv-1/runs/ebb31d91810c4b62d2b55b5382e8c7ea/logs?workspace=user-jxmorris12
checkpoint_folder = '/home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea'
args_str = '--per_device_train_batch_size 128 --per_device_eval_batch_size 128 --max_seq_length 32 --model_name_or_path t5-base --embedder_model_name gtr_base --num_repeat_tokens 16 --embedder_no_grad True --exp_group_name mar17-baselines --learning_rate 0.0003 --freeze_strategy none --embedder_fake_with_zeros False --use_frozen_embeddings_as_input False --num_train_epochs 24 --max_eval_samples 500 --eval_steps 25000 --warmup_steps 100000 --bf16=1 --use_wandb=1'
trainer = analyze_utils.load_inversion_model_and_trainer(checkpoint_folder, args_str)

trainer.evaluate()

[0] Loading model from checkpoint: /home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea/checkpoint-999744
Set train_args.dataloader_num_workers = 4
[1] creating model & stuff


/home/jxm3/.conda/envs/torch/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


[2] tokenizing dataset & preprocessing embeddings
[3] initializing trainer
[4] getting ckpnt
[5] loading ckpt /home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea/checkpoint-999744


generating from val:   0%|                                                                                           | 0/3 [00:00<?, ?it/s]

generate() -- embedder_decode_score False


...

generate() -- embedder_decode_score False


to the character of the skull, which are relatively smooth and untutored in the case of infant sutures. Nevertheless, the sutures
to the character of the sutures of the skull which, like those of the infant skull, are relatively smooth and untortuous. In fact



individual from the Southern Hemisphere to win the Winter Olympic relay gold medal, and was also part of the Australian Short Track team which won a
individual from the Southern Hemisphere, to win a Winter Olympic gold medal and was also part of the short track relay team that won Australia'



the same rights as men, and 75% agreed that they should be protected from discrimination. Among the other 15% are heterosexuals, no work
the same rights as straight people, while 15% disagreed. Additionally, 69% agreed that they should be protected from workplace discrimination. 13% of H


generating from train:   0%|                                                                                     | 0/41656 [00:00<?, ?it/s]

generate() -- embedder_decode_score False


{'eval_loss': 1.0522613525390625,
 'eval_bleu_score': 31.552541624779003,
 'eval_accuracy': 0.7386067708333334,
 'eval_perplexity': 2.8641205867926014,
 'eval_runtime': 5.4294,
 'eval_samples_per_second': 92.091,
 'eval_steps_per_second': 0.737}

## checking with sampling

In [3]:
batch = next(iter(trainer.get_eval_dataloader()))
true_text = trainer.model.embedder_tokenizer.batch_decode(batch['embedder_input_ids'], skip_special_tokens=True)

In [8]:
from sentence_transformers import SentenceTransformer
dpr_model = SentenceTransformer('nq-distilbert-base-v1')

In [27]:
import torch
with torch.no_grad():
    # embed
    dpr_passage_embedding = torch.tensor(dpr_model.encode(true_text), device=trainer.args.device)
    # project

    embeddings = trainer.model.embedding_transform(dpr_passage_embedding)
    batch_size = len(embeddings)
    # linear outputs a big embedding, reshape into a sequence of regular size embeddings.
    embeddings = embeddings.reshape((batch_size, trainer.model.num_repeat_tokens, -1))
attention_mask = torch.ones((batch_size, trainer.model.num_repeat_tokens), device=trainer.args.device)

In [28]:
dpr_passage_embedding.shape, attention_mask.shape

(torch.Size([128, 768]), torch.Size([128, 16]))

In [29]:
import numpy as np
import torch
import tqdm

torch.set_grad_enabled(False)
gen_kwargs = trainer.gen_kwargs

def gen(inputs_embeds: torch.Tensor, attention_mask: torch.Tensor):
    return trainer.model.encoder_decoder.generate(
        inputs_embeds=inputs_embeds,
        attention_mask=attention_mask,
        **gen_kwargs,
    )

t = gen(embeddings, attention_mask)
trainer.model.embedder_tokenizer.batch_decode(t)

['<pad> the the people people people people people the the   the the people people people people people people',
 '<pad> of sabotaging after-hours of sex work on left-handed',
 '<pad> nnnnnnnn s    nnnn',
 '<pad> a little bit of a hurry, when <unk>/ref> Motorola proprietary audio',
 '<pad> — / — — — — /  —  ',
 '<pad>————————————————— little little',
 '<pad> “The robot’s” side of the movie, where the doctor’s lab is ',
 '<pad> a drug-related institution in the United States: "Automobile, where someone named Bruno',
 '<pad>  your        specific specific specific    your  favorite favorite',
 '<pad> last last last last on crossing crossing crossing crossing crossing crossing crossing crossing crossing crossing  recently recently recently',
 "<pad> French slang word-based savage chicken coop heater's signature rock",
 '<pad> the the the  who who who who who who who who who who who who who who who',
 '<pad> ghetto-abanda naive BCE ghe',
 '<pad> after after after after after after first fi

In [26]:
true_text

['to the character of the sutures of the skull which, like those of the infant skull, are relatively smooth and untortuous. In fact',
 "individual from the Southern Hemisphere, to win a Winter Olympic gold medal and was also part of the short track relay team that won Australia'",
 'the same rights as straight people, while 15% disagreed. Additionally, 69% agreed that they should be protected from workplace discrimination. 13% of H',
 'Bruthen, Victoria Bruthen is a small town located alongside the Tambo River between Bairnsdale and Ensay on',
 'Castle Vale Castle Vale is a housing estate located between Erdington, Minworth and Castle Bromwich. Currently Castle Vale votes',
 '1839. After her death, Fisk married her sister, Simona, on August 1, 1849. Deaf Smith moved freely between Anglo',
 "in 1978 he was a member of the Soviet Composer's Society in Moscow. He obtained commissions from the Russian Broadcasting Corporation",
 'Zachary Throne Zachary Throne (born April 3, 1967 in Hollywo